<a href="https://colab.research.google.com/github/AbiramiRathina/roi_based_program_selection/blob/abirami/project_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import numpy as np

In [125]:
pd.set_option('display.max_rows', 5000) #Since we have alot of columns to analyse, setting this window to be higher than usual

# Dataset Description
*Dataset documentation: https://collegescorecard.ed.gov/data/data-documentation/*

*Dataset: https://collegescorecard.ed.gov/data/*

### Overview
In our project, we use two datasets: an institution-level dataset and a cohort (field-of-study)–level dataset. Combined, these data sources help us estimate the return on investment (ROI) of academic programs. Given a list of university choices, a student will be able to compare multiple programs based on ROI.

For simplicity, we restrict our analysis to data from the year 2025. Although we recognize that a fully informed decision requires examining trends across multiple years, the large dataset size and limitations in computational resources make multi-year analysis difficult. Therefore, we proceed with only the 2025 data. The first step in our project is to understand the structure and content of the data.

Since both datasets are large, instead of mounting Google Drive in Colab, we host them via public Google Drive links and load them directly. This approach saves time and ensures that the data is easily accessible to anyone running the project.

### Dataset 1: Institution-Level Data

This dataset contains information on approximately 6,429 institutions and 3,306 features. The data can be grouped into the following categories:

(i) Institutional Demographics

Institution name, location, control (public/private), sector

Campus type, degree levels offered

Admissions information, acceptance rates

Program offerings (CIP codes)

(ii) Cost & Affordability Indicators

Tuition and fees (in-state, out-of-state)

Net price after grants

Average annual cost by family income bracket

Cost of attendance and living expenses

(iii) Financial Aid & Debt

Percentage of students receiving Pell Grants

Average student loan amounts

Median debt at graduation

Repayment and default rates

(iv) Student Demographics

Enrollment numbers

Gender and race/ethnicity distributions

First-generation status

Part-time vs. full-time enrollment

(v) Academic Performance

Retention rates

Completion and graduation rates

Transfer-out and withdrawal rates

(vi) Earnings & Outcomes

Median earnings 1, 2, 6, and 10 years after entry

Employment rates

Loan repayment progress

Earnings by program or award level

Important notes:

Several fields contain NULL or privacy-suppressed values (e.g., “PrivacySuppressed”).

Some variables are only available for specific years.

Certain earnings metrics lag by several years due to tax data availability.

Proper interpretation requires understanding cohort definitions (e.g., first-time students, completers, non-completers).

### Dataset 2: Field-of-Study (Cohort-Level) Data

This dataset contains approximately 229,188 rows and 174 columns. Although the number of columns is smaller than in the institution dataset, the dimensionality remains significant. After reviewing the documentation, the following column groups help structure and understand the data:

(i) Identification & Keys

Institution-level identifiers
Program identifiers (CIP code, credential level)

(ii) Academic Program Information

Characteristics of the program or field of study

(iii) Student Count & Cohort Size

(iv) Cost & Tuition Information

(v) Debt, Loan & Repayment Metrics

(vi) Earnings & Employment Outcomes

(vii) Loan Repayment & Default Indicators

Useful for assessing program-level financial risk

(viii) Demographics

(ix) Program-level demographic details (gender, race, etc.)

(x) Institution Characteristics

Helpful when merging with the institution-level dataset

As with the institution data, this dataset contains many privacy-suppressed values (PS) and null or missing entries that require cleaning.

### EDA

#### Dataset 1: Institute level data

In [126]:
url_institutes = "https://drive.google.com/uc?export=download&id=1SIZufYNWCC91scwafSx3LYOAMEHyfvrr" # https://drive.google.com/file/d/1SIZufYNWCC91scwafSx3LYOAMEHyfvrr/view?usp=sharing"
df_institutes = pd.read_csv(url_institutes)

/tmp/ipython-input-1537099505.py:2: DtypeWarning: Columns (9,1407,1408,1431,1432,1532,1537,1538,1539,1540,1542,1546,1589,1601,1602,1606,1608,1611,1614,1615,1616,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1653,1679,1690,1692,1697,1700,1702,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,2958,3215,3231,3235,3236) have mixed types. Specify dtype option on import or set low_memory=False.
  df_institutes = pd.read_csv(url_institutes)


In [127]:
df_institutes

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE1_P11,GT_THRESHOLD_P11,MD_EARN_WNE_INC1_P11,MD_EARN_WNE_INC2_P11,MD_EARN_WNE_INC3_P11,MD_EARN_WNE_INDEP0_P11,MD_EARN_WNE_INDEP1_P11,MD_EARN_WNE_MALE0_P11,MD_EARN_WNE_MALE1_P11,SCORECARD_SECTOR
0,100654,100200.0,1002.0,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,...,777.0,0.6250,36650.0,41070.0,47016.0,38892.0,41738.0,38167.0,40250.0,4
1,100663,105200.0,1052.0,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,...,1157.0,0.7588,47182.0,51896.0,54368.0,50488.0,51505.0,46559.0,59181.0,4
2,100690,2503400.0,25034.0,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,https://www.amridgeuniversity.edu/,https://www2.amridgeuniversity.edu:9091/,...,67.0,0.5986,35752.0,41007.0,NaN,NaN,38467.0,32654.0,49435.0,5
3,100706,105500.0,1055.0,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,...,802.0,0.7810,51208.0,62219.0,62577.0,55920.0,60221.0,47787.0,67454.0,4
4,100724,100500.0,1005.0,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,...,1049.0,0.5378,32844.0,36932.0,37966.0,34294.0,31797.0,32303.0,36964.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424,49382201,4283601.0,42836.0,College Unbound - Newport/Aquidneck Island,Newport,RI,028400000,New England Commission on Higher Education,https://www.collegeunbound.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
6425,49425001,2609404.0,26094.0,Valley College - Fairlawn - School of Nursing,Fairlawn,OH,443333631,Accrediting Commission of Career Schools and C...,https://www.valley.edu/,NaN,...,NaN,0.4651,26087.0,37545.0,NaN,NaN,28205.0,27499.0,NaN,15
6426,49501301,4247201.0,42472.0,Western Maricopa Education Center - Southwest ...,Buckeye,AZ,85326-5705,Council on Occupational Education,https://west-mec.edu/findyourhappy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
6427,49501302,4247202.0,42472.0,Western Maricopa Education Center - Northeast ...,Phoenix,AZ,85027-0000,Council on Occupational Education,https://west-mec.edu/findyourhappy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13


In [128]:
df_institutes.shape

(6429, 3306)

As we can see we have 6429 rows and 3306 columns, this is alot of features, for our problem statement information regarding the strength of the program(value it holds in terms of roi) is what trully matters

In [129]:
df_institutes.dtypes.unique()


array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

we can see tat we have both numeric and object datatypes, that means we will have to extract stats for each differently

First we'll describle information about the numeric columns

In [130]:
df_institutes.describe().T

,count,mean,std,min,25%,50%,75%,max
UNITID,6429.0,2.355496e+06,8.043861e+06,100654.000000,1.745700e+05,2.295400e+05,4.589550e+05,4.966450e+07
OPEID,6405.0,1.762653e+06,2.173129e+06,100200.000000,3.048000e+05,1.019817e+06,3.101900e+06,8.209882e+07
OPEID6,6405.0,1.714251e+04,1.533956e+04,1002.000000,3.037000e+03,1.019800e+04,3.098700e+04,4.309800e+04
SCH_DEG,5926.0,1.961019e+00,9.019431e-01,1.000000,1.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00
HCM2,6429.0,5.599627e-03,7.462665e-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
MAIN,6429.0,7.883030e-01,4.085429e-01,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
NUMBRANCH,6429.0,3.183699e+00,6.591915e+00,1.000000,1.000000e+00,1.000000e+00,2.000000e+00,5.200000e+01
PREDDEG,6429.0,1.823612e+00,1.086918e+00,0.000000,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00
HIGHDEG,6429.0,2.247006e+00,1.388220e+00,0.000000,1.000000e+00,2.000000e+00,4.000000e+00,4.000000e+00
CONTROL,6429.0,2.056618e+00,8.325397e-01,1.000000,1.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00


In [131]:
df_institutes.describe(include="object").T

,count,unique,top,freq
INSTNM,6429,6321,Cortiva Institute,6
CITY,6429,2362,New York,75
STABBR,6429,59,CA,672
ZIP,6429,5819,00961,6
ACCREDAGENCY,6225,40,Higher Learning Commission,1153
INSTURL,6415,5541,www.empire.edu/,71
NPCURL,5887,5085,www.empire.edu/net-price-calculator,69
DEATH_YR2_RT,6183,25,PS,5945
COMP_ORIG_YR2_RT,6183,3337,PS,1228
COMP_4YR_TRANS_YR2_RT,6183,938,PS,4659


In [132]:
df_institutes.nunique().sort_values(ascending=False)

,0
UNITID,6429
OPEID,6377
INSTNM,6321
LATITUDE,5874
LONGITUDE,5873
ADDR,5868
ZIP,5819
INSTURL,5541
FEDSCHCD,5419
TUITFTE,5204


we definately have a lot of data, that just cant be manually double checked

Possible steps to reduce thew number of features for numeric columns is to first drop columns with null values, then we can also drop columns that have only one unique value, we seem to have a few of those.

For categorical data we can observe that the top value for alot of columns is PS(meaning these are not disclosed for privacy reasons, that means we might have to drop these columns as they don't hold much value)

we also see that with regards to unique values we have multiple identifies for institues but the UNITID and INSTNM are enough identifiers. There also many columns that have 0-1 unique values, these dont add any value to our model

In [133]:
empty_cols = df_institutes.columns[df_institutes.isna().all()]
len(empty_cols)

75

In [134]:
constant_cols = df_institutes.columns[df_institutes.nunique(dropna=True) <= 1]
len(constant_cols)

193

In [135]:
obj_desc = df_institutes.describe(include='object').T
ps_cols = obj_desc[obj_desc['top'] == 'PS'].index.tolist()
len(ps_cols)

2319

In [136]:
bad_cols = set(empty_cols) | set(constant_cols) | set(ps_cols)
len(bad_cols)

2400

we have 2400 columns that can be removed

In [137]:
df_institutes_cleaned = df_institutes.drop(columns=list(bad_cols))
df_institutes_cleaned.shape


(6429, 906)

In [138]:
df_institutes.shape

(6429, 3306)

We have successfully reduced the number of featues from 3306 to 906

In [139]:
df_institutes_cleaned.nunique().sort_values(ascending=False)

,0
UNITID,6429
OPEID,6377
INSTNM,6321
LATITUDE,5874
LONGITUDE,5873
ADDR,5868
ZIP,5819
INSTURL,5541
FEDSCHCD,5419
TUITFTE,5204


From the abouve table we can see that there are some columns that provide too much information that might not be needed like:OPEID(Office of Postsecondary Education Identifier), LATITUDE, LONGITUDE, ADDT, ZIP, FEDSCHCD (A type of fedral aid code), INSTURL, TUITFTE, NPCURL, OPEID6, INEXPFTE

And some columns related to demografic information, that dont hold much value for our problem. We can remove these

Any column with cip in the begining again dont matter much here as these are course level completion rates, this is not required for our project

Columns tha have HH in it, are household information, again of very less value in our project context, eg: LN_MEDIAN_HH_INC

POOLYRS* columns only tell you how many years of data were pooled to calculate certain repayment variables. They are metadata, not features. They do not help with prediction and do not describe the institution or program. Examples of such columns:POOLYRS100, POOLYRS1, POOLYRS10, POOLYRS5

We can remove MTHCMP1 and similar columns (MTHCMP2 … MTHCMP6). These columns only describe the average months to complete the institution’s top programs, not the specific program you’re evaluating. They do not contribute to ROI modeling and mostly add noise, so dropping them is appropriate.

In [140]:
cols_to_remove_manual = [
    'OPEID', 'LATITUDE', 'LONGITUDE', 'ADDR', 'ZIP', 'FEDSCHCD',
    'INSTURL', 'TUITFTE', 'NPCURL', 'OPEID6', 'INEXPFTE'
]

df_institutes_cleaned = df_institutes_cleaned.drop(columns=[col for col in cols_to_remove_manual if col in df_institutes_cleaned.columns])


In [141]:
df_institutes_cleaned.shape

(6429, 895)

In [142]:
demographic_keywords = [
    "male", "female", "men", "women",
    "black", "white", "hisp", "asian",
    "race", "ethnic", "minority", "cip", "hh", "poolyrs", "mthcmp"
]

demographic_cols = [
    c for c in df_institutes_cleaned.columns
    if any(k in c.lower() for k in demographic_keywords)
]

print("Columns to remove:", demographic_cols)


Columns to remove: ['MENONLY', 'WOMENONLY', 'PCIP01', 'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12', 'PCIP13', 'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24', 'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38', 'PCIP39', 'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44', 'PCIP45', 'PCIP46', 'PCIP47', 'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51', 'PCIP52', 'PCIP54', 'CIP01CERT1', 'CIP01CERT2', 'CIP01ASSOC', 'CIP01CERT4', 'CIP01BACHL', 'CIP03CERT1', 'CIP03CERT2', 'CIP03ASSOC', 'CIP03CERT4', 'CIP03BACHL', 'CIP04CERT1', 'CIP04CERT2', 'CIP04ASSOC', 'CIP04CERT4', 'CIP04BACHL', 'CIP05CERT1', 'CIP05CERT2', 'CIP05ASSOC', 'CIP05CERT4', 'CIP05BACHL', 'CIP09CERT1', 'CIP09CERT2', 'CIP09ASSOC', 'CIP09CERT4', 'CIP09BACHL', 'CIP10CERT1', 'CIP10CERT2', 'CIP10ASSOC', 'CIP10CERT4', 'CIP10BACHL', 'CIP11CERT1', 'CIP11CERT2', 'CIP11ASSOC', 'CIP11CERT4', 'CIP11BACHL', 'CIP12CERT1', 'CIP12CERT2', 'CIP12ASSOC', 'CIP12CERT4', 'CIP12BACHL', 'CIP13CERT1', 'C

In [143]:
len(set(demographic_cols))

464

In [144]:
df_institutes_cleaned = df_institutes_cleaned.drop(columns=[col for col in demographic_cols if col in df_institutes_cleaned.columns])


In [145]:
df_institutes_cleaned.shape

(6429, 431)

In [146]:
df_institutes_cleaned.nunique().sort_values(ascending=False)

,0
UNITID,6429
INSTNM,6321
MD_EARN_WNE_1YR,4116
MD_EARN_WNE_5YR,4072
PCTFLOAN_DCS_POOLED_SUPP,4045
MD_EARN_WNE_4YR,4028
PCT75_EARN_WNE_P6,4005
MD_EARN_WNE_P6,3986
PCT75_EARN_WNE_P8,3962
MD_EARN_WNE_P8,3959


we have done alot of fikltering to reduce the number of features, ut we just dont seem to get the right number of columns, hence we are going to use a tree based model like RandomForestRegressor to pick the top 20 features. The ability to prune these tree models, can be a useful technique in our case to select important features, lets assume try to do this but trying to predict MD_EARN_WNE_1YR

In [147]:
df_institutes_cleaned_features_select = df_institutes_cleaned.dropna(subset=['MD_EARN_WNE_1YR'])


In [148]:
y = df_institutes_cleaned_features_select['MD_EARN_WNE_1YR']
X = df_institutes_cleaned_features_select.drop(columns=['MD_EARN_WNE_1YR'])

In [149]:
X = X.select_dtypes(include=['float64','int64'])

In [150]:
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

In [151]:
model = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

model.fit(X_imputed, y)


RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)

In [152]:
importances = pd.Series(model.feature_importances_, index=X.columns)
important_cols = importances.sort_values(ascending=False)
important_cols.head(30)

,0
MD_EARN_WNE_4YR,0.709182
MD_EARN_WNE_5YR,0.054804
SCORECARD_SECTOR,0.016385
PREDDEG,0.012938
MD_EARN_WNE_P6,0.009218
UG25ABV,0.007279
NPT4_PRIV,0.006489
MD_EARN_WNE_INC2_P6,0.005350
COUNT_NWNE_1YR,0.004636
COUNT_WNE_1YR,0.004507


We now have 30 features from which we can now easily handpick:
lets choose the following columns

UNITID: for unique identification of universities

INSTNM: Ewven though it was not chosen by the model, this represents the name of the university in text, and can be usful while interpreting results

1. MD_EARN_WNE_4YR

Median earnings of completers who are working and not enrolled 4 years after graduation.

2. MD_EARN_WNE_5YR

Median earnings of completers 5 years after graduation (long-term earnings signal).

3. MD_EARN_WNE_P6

60th-percentile earnings of graduates (captures upper-middle earning potential).

4. GT_THRESHOLD_1YR

Share of graduates earning above the federal threshold one year after completion.

5. GT_THRESHOLD_P6

Percentage of graduates hitting the earnings threshold at the 60th percentile.

6. TUITIONFEE_PROG

Program-specific tuition and fees (direct cost impacting ROI).

7. NPT4_PRIV

Net price for private 4-year institutions after grants (actual cost to student).

8. MD_EARN_WNE_INC2_P6

60th-percentile earnings of graduates from middle-income families (income group 2).

9. MD_EARN_WNE_INC3_P6

60th-percentile earnings of graduates from higher-income families (income group 3).

10. PCT75_EARN_WNE_P11

Percentage of graduates reaching threshold earnings at the 75th percentile.

11. ROOMBOARD_ON

Average on-campus room and board cost (major part of total cost).

12. AVGFACSAL

Average faculty salary (proxy for institutional quality and resources).



In [153]:
selected_features = [
    "UNITID",
    "INSTNM",
    "MD_EARN_WNE_4YR",
    "MD_EARN_WNE_5YR",
    "MD_EARN_WNE_P6",
    "GT_THRESHOLD_1YR",
    "GT_THRESHOLD_P6",
    "TUITIONFEE_PROG",
    "NPT4_PRIV",
    "MD_EARN_WNE_INC2_P6",
    "MD_EARN_WNE_INC3_P6",
    "PCT75_EARN_WNE_P11",
    "ROOMBOARD_ON",
    "AVGFACSAL"
]

df_institutes_cleaned_final = df_institutes_cleaned[selected_features]


SELECTED FEATURE DESCRIPTION

UNITID: Unique institutional identifier assigned by the U.S. Department of Education (IPEDS).

INSTNM: Official name of the institution as reported to IPEDS.

MD_EARN_WNE_4YR: Median earnings of graduates who are working and not enrolled four years after completion.

MD_EARN_WNE_5YR: Median earnings of graduates who are working and not enrolled five years after completion.

MD_EARN_WNE_P6: 60th-percentile earnings of graduates who are working and not enrolled.

GT_THRESHOLD_1YR: Share of graduates earning above the federal threshold level one year after completion.

GT_THRESHOLD_P6: Percentage of graduates exceeding the threshold earnings level at the 60th percentile.

TUITIONFEE_PROG: Program-specific tuition and required fees for the institution.

NPT4_PRIV: Average net price after grants for private institutions offering four-year programs.

MD_EARN_WNE_INC2_P6: 60th-percentile earnings of graduates from middle-income families (income group 2).

MD_EARN_WNE_INC3_P6: 60th-percentile earnings of graduates from higher-income families (income group 3).

PCT75_EARN_WNE_P11: Percentage of graduates meeting threshold earnings at the 75th percentile in comparison group 11.

ROOMBOARD_ON: Average annual on-campus room and board charges.

AVGFACSAL: Average faculty salary at the institution, reflecting instructional quality and resources.


In [154]:
df_institutes_cleaned_final

,UNITID,INSTNM,MD_EARN_WNE_4YR,MD_EARN_WNE_5YR,MD_EARN_WNE_P6,GT_THRESHOLD_1YR,GT_THRESHOLD_P6,TUITIONFEE_PROG,NPT4_PRIV,MD_EARN_WNE_INC2_P6,MD_EARN_WNE_INC3_P6,PCT75_EARN_WNE_P11,ROOMBOARD_ON,AVGFACSAL
0,100654,Alabama A & M University,46562.0,52246.0,27851.0,355.0,0.4613,NaN,NaN,31228.0,33539.0,56598.0,11402.0,8610.0
1,100663,University of Alabama at Birmingham,52404.0,60738.0,46572.0,2290.0,0.7443,NaN,NaN,49623.0,50532.0,75896.0,13590.0,12211.0
2,100690,Amridge University,45765.0,49649.0,30377.0,20.0,0.5026,NaN,NaN,NaN,NaN,59803.0,NaN,5109.0
3,100706,University of Alabama in Huntsville,67695.0,78740.0,55610.0,880.0,0.7854,NaN,NaN,56719.0,60565.0,87130.0,11122.0,10411.0
4,100724,Alabama State University,37551.0,43913.0,27453.0,316.0,0.4467,NaN,NaN,28989.0,31482.0,49594.0,7690.0,8015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424,49382201,College Unbound - Newport/Aquidneck Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6425,49425001,Valley College - Fairlawn - School of Nursing,22873.0,25262.0,25548.0,136.0,0.4118,18625.0,NaN,NaN,NaN,40067.0,NaN,NaN
6426,49501301,Western Maricopa Education Center - Southwest ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6427,49501302,Western Maricopa Education Center - Northeast ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
df_institutes_cleaned_final.describe().T

,count,mean,std,min,25%,50%,75%,max
UNITID,6429.0,2.355496e+06,8.043861e+06,100654.0000,174570.000000,229540.0,458955.0000,49664501.0
MD_EARN_WNE_4YR,5500.0,4.066770e+04,1.676042e+04,9253.0000,28120.750000,39686.5,49565.0000,139418.0
MD_EARN_WNE_5YR,5484.0,4.620659e+04,1.923309e+04,8049.0000,31471.000000,45490.0,56504.5000,160672.0
MD_EARN_WNE_P6,5450.0,3.729980e+04,1.445920e+04,8535.0000,27583.000000,35565.5,44256.0000,143353.0
GT_THRESHOLD_1YR,5041.0,9.256985e+02,1.988892e+03,16.0000,77.000000,225.0,757.0000,27755.0
GT_THRESHOLD_P6,5180.0,5.949676e-01,1.671113e-01,0.1467,0.474625,0.6,0.7241,1.0
TUITIONFEE_PROG,2154.0,1.731196e+04,1.045951e+04,585.0000,12438.250000,16297.0,19260.0000,157200.0
NPT4_PRIV,3360.0,2.137254e+04,9.090793e+03,1124.0000,15549.250000,20773.5,26094.5000,112070.0
MD_EARN_WNE_INC2_P6,3773.0,4.409908e+04,1.263384e+04,11779.0000,36412.000000,41967.0,50137.0000,141916.0
MD_EARN_WNE_INC3_P6,3773.0,4.708219e+04,1.403420e+04,12082.0000,38650.000000,45153.0,53417.0000,147468.0


In [156]:
df_institutes_cleaned_final.describe(include="object").T

,count,unique,top,freq
INSTNM,6429,6321,Cortiva Institute,6


In [157]:
df_institutes_cleaned_final.nunique().sort_values(ascending=False)

,0
UNITID,6429
INSTNM,6321
MD_EARN_WNE_5YR,4072
MD_EARN_WNE_4YR,4028
MD_EARN_WNE_P6,3986
PCT75_EARN_WNE_P11,3695
NPT4_PRIV,3149
AVGFACSAL,3123
GT_THRESHOLD_P6,2805
MD_EARN_WNE_INC3_P6,2592


Now our institues, data looks almost perfect, but for now we are going to stop here, and not reduce the dimentions further, as we'll also have to extract important features from the cohort level information ,merge both tables and produce a final dataset where we will do a final feature selection to confirm the columns most useful

## Lets do the same thing for our cohorts dataset

In [158]:
url_cohorts = "https://drive.usercontent.google.com/download?id=1dkZwR3JDSTpH9j3oETqIO2L90WSHDgbT&export=download&confirm=t" #https://drive.google.com/file/d/1dkZwR3JDSTpH9j3oETqIO2L90WSHDgbT/view?usp=sharing
df_cohorts = pd.read_csv(url_cohorts)

In [159]:
df_cohorts

,UNITID,OPEID6,INSTNM,CONTROL,MAIN,CIPCODE,CIPDESC,CREDLEV,CREDDESC,IPEDSCOUNT1,...,EARN_COUNT_PELL_WNE_5YR,EARN_PELL_WNE_MDN_5YR,EARN_COUNT_NOPELL_WNE_5YR,EARN_NOPELL_WNE_MDN_5YR,EARN_COUNT_MALE_WNE_5YR,EARN_MALE_WNE_MDN_5YR,EARN_COUNT_NOMALE_WNE_5YR,EARN_NOMALE_WNE_MDN_5YR,EARN_COUNT_HIGH_CRED_5YR,EARN_IN_STATE_5YR
0,100654.0,1002,Alabama A & M University,Public,1,100,"Agriculture, General.",3,Bachelor's Degree,NaN,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
1,100654.0,1002,Alabama A & M University,Public,1,101,Agricultural Business and Management.,3,Bachelor's Degree,NaN,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
2,100654.0,1002,Alabama A & M University,Public,1,109,Animal Sciences.,3,Bachelor's Degree,3.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
3,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,3,Bachelor's Degree,7.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
4,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,5,Master's Degree,4.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229183,NaN,43006,Southeast New Mexico College,Public,1,5201,"Business/Commerce, General.",2,Associate's Degree,NaN,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
229184,NaN,43006,Southeast New Mexico College,Public,1,5203,Accounting and Related Services.,1,Undergraduate Certificate or Diploma,NaN,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
229185,NaN,43006,Southeast New Mexico College,Public,1,5204,Business Operations Support and Assistant Serv...,1,Undergraduate Certificate or Diploma,NaN,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
229186,NaN,43006,Southeast New Mexico College,Public,1,5204,Business Operations Support and Assistant Serv...,2,Associate's Degree,NaN,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS


Lets drop nan values

In [160]:
df_cohorts=df_cohorts.dropna()

In [161]:
df_cohorts.shape

(171410, 174)

So we have more rows and lesser columns this time, we'll have to reduce the dimentionality ti maybe 20 features first

In [162]:
df_cohorts.describe().T

,count,mean,std,min,25%,50%,75%,max
UNITID,171410.0,199684.860539,84132.109461,100654.0,148016.0,186371.0,219709.0,496371.0
OPEID6,171410.0,5805.593612,8172.313385,1002.0,1949.0,3009.0,4586.0,43006.0
MAIN,171410.0,0.948469,0.221080,0.0,1.0,1.0,1.0,1.0
CIPCODE,171410.0,3326.847634,1703.094246,100.0,1437.0,4005.0,5106.0,5401.0
CREDLEV,171410.0,3.243381,1.837564,1.0,2.0,3.0,5.0,8.0
IPEDSCOUNT1,171410.0,30.136404,100.750515,0.0,2.0,9.0,26.0,9082.0
IPEDSCOUNT2,171410.0,30.445995,105.566080,0.0,2.0,8.0,26.0,9606.0
DISTANCE,171410.0,1.258147,0.612038,1.0,1.0,1.0,1.0,3.0


Analysing the numeric columns is reletively easier compared to categorical features as they are less in number,

The UNITID is an important key as it will act as our foreign key mapping to UNITID in the institues table--> required

OPEID6 is not relevant in our case as it stands for Office of Postsecondary Education Identifier and this is not useful in our case-->not required

MAIN is a binary flag used to identify whether the campus is a main campus or a branch of the main campus/affiliated campus-->could be usefull

CIPCODE	is a numeric code assigned to a major can can be used along with the CIPDESC(name of the course) to identify majors-->we can remove this

CREDLEV tells you the type of degree earned for that program (certificate, associate, bachelor’s, master’s, or doctoral).-->we also have a CREDDESC which is a text description of the course, this is more easy for use to undersand, and we could get a similar encoding using one-hot latter while model building , hence for now we'll remove this

IPEDSCOUNT1: Number of students in the IPEDS Outcome Measures 8-year cohort (full-time, first-time undergraduates).

IPEDSCOUNT2: Number of students in the IPEDS Outcome Measures 8-year cohort (part-time, first-time undergraduates).

DISTANCE: Indicator for whether the institution primarily delivers education through distance (online) learning.--> not requied


Conclusion: lets remove OPEID6 and DISTANCE from our df_cohorts



In [163]:
df_cohortsdf_cohorts = df_cohorts.drop(columns=["OPEID6", "DISTANCE", "CIPCODE", "CREDLEV"], errors="ignore")

In [164]:
df_cohorts.describe(include="object").T

,count,unique,top,freq
INSTNM,171410,5755,The Pennsylvania State University,450
CONTROL,171410,3,Public,107350
CIPDESC,171410,370,"Business Administration, Management and Operat...",5186
CREDDESC,171410,7,Bachelor's Degree,59375
DEBT_ALL_STGP_ANY_N,171410,859,PS,124190
DEBT_ALL_STGP_ANY_MEAN,171410,17573,PS,138880
DEBT_ALL_STGP_ANY_MDN,171410,11835,PS,134282
DEBT_ALL_STGP_EVAL_N,171410,874,PS,114947
DEBT_ALL_STGP_EVAL_MEAN,171410,24985,PS,128962
DEBT_ALL_STGP_EVAL_MDN,171410,15879,PS,127672


We see that alot of columns have the value PS, meaning private information that has'nt been disclosed, this might not be usefull in out case, but we can't completly remove them as we'll hance we must come up with a better solution

Lets first try to coun the number of non PS values and how they look like

In [165]:
is_ps = df_cohorts.isin(["PS"])
ps_counts = is_ps.sum()

total_rows = len(df_cohorts)

non_ps_counts = total_rows - ps_counts

dtypes = df_cohorts.dtypes

summary = pd.DataFrame({
    "dtype": dtypes,
    "total_rows": total_rows,
    "ps_count": ps_counts,
    "non_ps_count": non_ps_counts
})

summary.sort_values("ps_count", ascending=False)

,dtype,total_rows,ps_count,non_ps_count
DEBT_MALE_PP_EVAL_MDN,object,171410,169886,1524
DEBT_NOTMALE_PP_EVAL_MDN,object,171410,169886,1524
EARN_COUNT_HIGH_CRED_1YR,object,171410,169711,1699
DEBT_MALE_PP_ANY_MDN,object,171410,169431,1979
DEBT_NOTMALE_PP_ANY_MDN,object,171410,169431,1979
DEBT_NOPELL_PP_EVAL_MDN,object,171410,168974,2436
DEBT_PELL_PP_EVAL_MDN,object,171410,168974,2436
DEBT_MALE_PP_EVAL_MEAN,object,171410,168719,2691
DEBT_NOTMALE_PP_EVAL_MEAN,object,171410,168685,2725
DEBT_NOPELL_PP_ANY_MDN,object,171410,168254,3156


Now we have aorund 200k rows but for our project we need min of 50k rows, we see that for some cols the number of non PS values are cloese to this 50k range, lets filter them

Our goal is to use features related to debt, earnings and other related features that can help predict roi of a course


Lets try to manually pick out the cols that have more than 50k non_ps values, and check the size of the data

In [166]:
df_cohorts

,UNITID,OPEID6,INSTNM,CONTROL,MAIN,CIPCODE,CIPDESC,CREDLEV,CREDDESC,IPEDSCOUNT1,...,EARN_COUNT_PELL_WNE_5YR,EARN_PELL_WNE_MDN_5YR,EARN_COUNT_NOPELL_WNE_5YR,EARN_NOPELL_WNE_MDN_5YR,EARN_COUNT_MALE_WNE_5YR,EARN_MALE_WNE_MDN_5YR,EARN_COUNT_NOMALE_WNE_5YR,EARN_NOMALE_WNE_MDN_5YR,EARN_COUNT_HIGH_CRED_5YR,EARN_IN_STATE_5YR
2,100654.0,1002,Alabama A & M University,Public,1,109,Animal Sciences.,3,Bachelor's Degree,3.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
3,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,3,Bachelor's Degree,7.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
4,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,5,Master's Degree,4.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
5,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,6,Doctoral Degree,2.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
7,100654.0,1002,Alabama A & M University,Public,1,199,"Agriculture, Agriculture Operations, and Relat...",3,Bachelor's Degree,6.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218846,496283.0,23608,Provo College-Idaho Falls Campus,"Private, for-profit",0,5138,"Registered Nursing, Nursing Administration, Nu...",3,Bachelor's Degree,0.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
218849,496292.0,23068,Miller-Motte College-Chattanooga 2,"Private, for-profit",0,4902,Ground Transportation.,1,Undergraduate Certificate or Diploma,0.0,...,PS,PS,16,31181,PS,PS,PS,PS,PS,40
218850,496326.0,21785,Eagle Gate College-Boise Campus,"Private, for-profit",0,5108,Allied Health and Medical Assisting Services.,1,Undergraduate Certificate or Diploma,0.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,63
218851,496326.0,21785,Eagle Gate College-Boise Campus,"Private, for-profit",0,5138,"Registered Nursing, Nursing Administration, Nu...",3,Bachelor's Degree,0.0,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS


In [167]:
cols_to_extract = [
    "UNITID",
    "INSTNM",
    "DISTANCE",
    "IPEDSCOUNT1",
    "CIPDESC",
    "IPEDSCOUNT2",
    "CREDDESC",
    "CREDLEV",
    "OPEID6",
    "CONTROL",
    "MAIN",
    "CIPCODE",
    "EARN_COUNT_NWNE_1YR",
    "EARN_COUNT_NWNE_HI_1YR",
    "EARN_COUNT_NWNE_5YR",
    "EARN_COUNT_NWNE_4YR",
    "BBRR1_FED_COMP_N",
    "DEBT_ALL_STGP_EVAL_N",
    "BBRR2_FED_COMP_N",
    "BBRR4_FED_COMP_N",
    "BBRR3_FED_COMP_N",
    "EARN_CNTOVER150_HI_2YR",
    "EARN_COUNT_WNE_HI_2YR",
    "EARN_COUNT_NWNE_HI_2YR",
    "EARN_MDN_HI_2YR"
]

df_cohort_manual = df_cohorts[cols_to_extract]

In [168]:
df_cohort_manual

,UNITID,INSTNM,DISTANCE,IPEDSCOUNT1,CIPDESC,IPEDSCOUNT2,CREDDESC,CREDLEV,OPEID6,CONTROL,...,EARN_COUNT_NWNE_4YR,BBRR1_FED_COMP_N,DEBT_ALL_STGP_EVAL_N,BBRR2_FED_COMP_N,BBRR4_FED_COMP_N,BBRR3_FED_COMP_N,EARN_CNTOVER150_HI_2YR,EARN_COUNT_WNE_HI_2YR,EARN_COUNT_NWNE_HI_2YR,EARN_MDN_HI_2YR
2,100654.0,Alabama A & M University,1,3.0,Animal Sciences.,9.0,Bachelor's Degree,3,1002,Public,...,2,PS,PS,PS,PS,PS,PS,PS,PS,PS
3,100654.0,Alabama A & M University,1,7.0,Food Science and Technology.,10.0,Bachelor's Degree,3,1002,Public,...,0,PS,PS,PS,PS,PS,PS,PS,PS,PS
4,100654.0,Alabama A & M University,1,4.0,Food Science and Technology.,6.0,Master's Degree,5,1002,Public,...,0,PS,PS,15,12,12,PS,PS,PS,PS
5,100654.0,Alabama A & M University,1,2.0,Food Science and Technology.,4.0,Doctoral Degree,6,1002,Public,...,PS,PS,PS,PS,PS,PS,PS,PS,PS,PS
7,100654.0,Alabama A & M University,1,6.0,"Agriculture, Agriculture Operations, and Relat...",6.0,Bachelor's Degree,3,1002,Public,...,1,PS,PS,PS,PS,PS,PS,PS,PS,PS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218846,496283.0,Provo College-Idaho Falls Campus,1,0.0,"Registered Nursing, Nursing Administration, Nu...",0.0,Bachelor's Degree,3,23608,"Private, for-profit",...,PS,107,PS,11,PS,PS,PS,PS,PS,PS
218849,496292.0,Miller-Motte College-Chattanooga 2,1,0.0,Ground Transportation.,47.0,Undergraduate Certificate or Diploma,1,23068,"Private, for-profit",...,8,PS,18,PS,PS,PS,32,43,8,30988
218850,496326.0,Eagle Gate College-Boise Campus,1,0.0,Allied Health and Medical Assisting Services.,4.0,Undergraduate Certificate or Diploma,1,21785,"Private, for-profit",...,7,48,PS,117,97,96,12,17,2,25658
218851,496326.0,Eagle Gate College-Boise Campus,1,0.0,"Registered Nursing, Nursing Administration, Nu...",0.0,Bachelor's Degree,3,21785,"Private, for-profit",...,0,100,108,11,PS,PS,PS,PS,PS,PS


lets drop rows with ps in it

In [169]:
df_cohort_manual_non_ps = df_cohort_manual[~df_cohort_manual.isin(["PS"]).any(axis=1)]


In [170]:
df_cohort_manual_non_ps

,UNITID,INSTNM,DISTANCE,IPEDSCOUNT1,CIPDESC,IPEDSCOUNT2,CREDDESC,CREDLEV,OPEID6,CONTROL,...,EARN_COUNT_NWNE_4YR,BBRR1_FED_COMP_N,DEBT_ALL_STGP_EVAL_N,BBRR2_FED_COMP_N,BBRR4_FED_COMP_N,BBRR3_FED_COMP_N,EARN_CNTOVER150_HI_2YR,EARN_COUNT_WNE_HI_2YR,EARN_COUNT_NWNE_HI_2YR,EARN_MDN_HI_2YR
15,100654.0,Alabama A & M University,1,23.0,Audiovisual Communications Technologies/Techni...,16.0,Bachelor's Degree,3,1002,Public,...,0,48,45,18,16,16,6,12,0,20102
16,100654.0,Alabama A & M University,1,20.0,"Computer and Information Sciences, General.",37.0,Bachelor's Degree,3,1002,Public,...,0,46,41,26,29,29,29,31,0,52107
28,100654.0,Alabama A & M University,1,18.0,Civil Engineering.,7.0,Bachelor's Degree,3,1002,Public,...,0,10,12,12,15,15,12,12,0,47260
29,100654.0,Alabama A & M University,1,33.0,"Electrical, Electronics and Communications Eng...",20.0,Bachelor's Degree,3,1002,Public,...,4,43,50,23,22,22,28,29,2,61303
30,100654.0,Alabama A & M University,1,41.0,Mechanical Engineering.,41.0,Bachelor's Degree,3,1002,Public,...,2,39,53,23,26,26,19,23,0,52107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218828,495998.0,San Joaquin Valley College-Porterville,1,82.0,Allied Health and Medical Assisting Services.,57.0,Undergraduate Certificate or Diploma,1,21207,"Private, for-profit",...,112,2097,2325,1311,1098,1090,491,797,84,21861
218829,495998.0,San Joaquin Valley College-Porterville,1,19.0,Business Operations Support and Assistant Serv...,11.0,Undergraduate Certificate or Diploma,1,21207,"Private, for-profit",...,17,258,306,135,113,113,36,60,11,22094
218844,496265.0,Triangle Tech-Chambersburg,1,0.0,Precision Metal Working.,1.0,Associate's Degree,2,7839,"Private, for-profit",...,7,25,20,33,54,54,35,50,5,30409
218845,496283.0,Provo College-Idaho Falls Campus,1,0.0,Allied Health and Medical Assisting Services.,4.0,Undergraduate Certificate or Diploma,1,23608,"Private, for-profit",...,3,30,22,55,35,35,9,13,0,23456


This is giving us only 36k rows, that means there are some rows, where there is only one PS value and the others are available, lets loosen this condition a bit and remove only rows that have more than 4 ps values

In [171]:
ps_count_per_row = df_cohort_manual.isin(["PS"]).sum(axis=1)

df_cohort_filtered = df_cohort_manual[ps_count_per_row <= 4]

In [172]:
df_cohort_filtered

,UNITID,INSTNM,DISTANCE,IPEDSCOUNT1,CIPDESC,IPEDSCOUNT2,CREDDESC,CREDLEV,OPEID6,CONTROL,...,EARN_COUNT_NWNE_4YR,BBRR1_FED_COMP_N,DEBT_ALL_STGP_EVAL_N,BBRR2_FED_COMP_N,BBRR4_FED_COMP_N,BBRR3_FED_COMP_N,EARN_CNTOVER150_HI_2YR,EARN_COUNT_WNE_HI_2YR,EARN_COUNT_NWNE_HI_2YR,EARN_MDN_HI_2YR
12,100654.0,Alabama A & M University,1,8.0,"City/Urban, Community and Regional Planning.",6.0,Master's Degree,5,1002,Public,...,0,PS,PS,PS,10,10,12,12,0,47260
15,100654.0,Alabama A & M University,1,23.0,Audiovisual Communications Technologies/Techni...,16.0,Bachelor's Degree,3,1002,Public,...,0,48,45,18,16,16,6,12,0,20102
16,100654.0,Alabama A & M University,1,20.0,"Computer and Information Sciences, General.",37.0,Bachelor's Degree,3,1002,Public,...,0,46,41,26,29,29,29,31,0,52107
17,100654.0,Alabama A & M University,3,9.0,"Computer and Information Sciences, General.",7.0,Master's Degree,5,1002,Public,...,0,10,11,13,12,12,PS,PS,PS,PS
20,100654.0,Alabama A & M University,1,1.0,Educational Administration and Supervision.,1.0,Master's Degree,5,1002,Public,...,2,PS,PS,15,20,20,8,11,0,50231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218844,496265.0,Triangle Tech-Chambersburg,1,0.0,Precision Metal Working.,1.0,Associate's Degree,2,7839,"Private, for-profit",...,7,25,20,33,54,54,35,50,5,30409
218845,496283.0,Provo College-Idaho Falls Campus,1,0.0,Allied Health and Medical Assisting Services.,4.0,Undergraduate Certificate or Diploma,1,23608,"Private, for-profit",...,3,30,22,55,35,35,9,13,0,23456
218849,496292.0,Miller-Motte College-Chattanooga 2,1,0.0,Ground Transportation.,47.0,Undergraduate Certificate or Diploma,1,23068,"Private, for-profit",...,8,PS,18,PS,PS,PS,32,43,8,30988
218850,496326.0,Eagle Gate College-Boise Campus,1,0.0,Allied Health and Medical Assisting Services.,4.0,Undergraduate Certificate or Diploma,1,21785,"Private, for-profit",...,7,48,PS,117,97,96,12,17,2,25658


This looks good enough, lets now impute missing PS values using SimpleImputed

In [173]:
df_cohort_filtered = df_cohort_filtered.replace(["PS"], np.nan) #sub PS values with nan

Since imputation must be done seperately for numeric and cat cols lets seperate them

In [174]:
numeric_cols = df_cohort_filtered.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_cohort_filtered.select_dtypes(exclude=[np.number]).columns.tolist()

In [175]:
num_imputer = SimpleImputer(strategy="median")
df_cohort_filtered[numeric_cols] = num_imputer.fit_transform(df_cohort_filtered[numeric_cols])

In [176]:
cat_imputer = SimpleImputer(strategy="most_frequent")
df_cohort_filtered[categorical_cols] = cat_imputer.fit_transform(df_cohort_filtered[categorical_cols])

Lets try another method to do this, we will use a kind of thresholding to strike a balance between the number of rows after removing columns with non_ps values above a threshold, our goal is to reduce the number of rows but not as many columns, because our data is spread across

Lets come up with a logic that finds the exact threshold in 100 increments where the final df after filetring is more than 50k rows, we want to keep maximum number of columns, so the threshold should be small enough to strike a balance between these two

In [177]:
start_threshold = 50000
end_threshold = 100000
threshold=start_threshold

for threshold in range(start_threshold, end_threshold + 1, 100):
  cols_to_keep = summary[summary["non_ps_count"] >= threshold].index.tolist()
  df_thresh = df_cohorts[cols_to_keep]
  df_thresh_non_ps = df_thresh[
    ~df_thresh.isin(["PS"]).any(axis=1)]
  if len(df_thresh_non_ps) >= 50000:
    break

In [178]:
threshold

56500

In [179]:
cols_to_keep

['UNITID',
 'OPEID6',
 'INSTNM',
 'CONTROL',
 'MAIN',
 'CIPCODE',
 'CIPDESC',
 'CREDLEV',
 'CREDDESC',
 'IPEDSCOUNT1',
 'IPEDSCOUNT2',
 'EARN_COUNT_NWNE_HI_1YR',
 'EARN_COUNT_NWNE_1YR',
 'EARN_COUNT_NWNE_4YR',
 'BBRR1_FED_COMP_N',
 'DISTANCE',
 'EARN_COUNT_NWNE_5YR']

In [180]:
df_thresh_non_ps

,UNITID,OPEID6,INSTNM,CONTROL,MAIN,CIPCODE,CIPDESC,CREDLEV,CREDDESC,IPEDSCOUNT1,IPEDSCOUNT2,EARN_COUNT_NWNE_HI_1YR,EARN_COUNT_NWNE_1YR,EARN_COUNT_NWNE_4YR,BBRR1_FED_COMP_N,DISTANCE,EARN_COUNT_NWNE_5YR
10,100654.0,1002,Alabama A & M University,Public,1,305,Forestry.,3,Bachelor's Degree,5.0,4.0,0,1,0,11,1,1
15,100654.0,1002,Alabama A & M University,Public,1,1002,Audiovisual Communications Technologies/Techni...,3,Bachelor's Degree,23.0,16.0,1,1,0,48,1,0
16,100654.0,1002,Alabama A & M University,Public,1,1101,"Computer and Information Sciences, General.",3,Bachelor's Degree,20.0,37.0,1,0,0,46,1,2
17,100654.0,1002,Alabama A & M University,Public,1,1101,"Computer and Information Sciences, General.",5,Master's Degree,9.0,7.0,1,1,0,10,3,1
21,100654.0,1002,Alabama A & M University,Public,1,1310,Special Education and Teaching.,3,Bachelor's Degree,18.0,12.0,0,2,0,17,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218844,496265.0,7839,Triangle Tech-Chambersburg,"Private, for-profit",0,4805,Precision Metal Working.,2,Associate's Degree,0.0,1.0,2,4,7,25,1,10
218845,496283.0,23608,Provo College-Idaho Falls Campus,"Private, for-profit",0,5108,Allied Health and Medical Assisting Services.,1,Undergraduate Certificate or Diploma,0.0,4.0,4,0,3,30,1,3
218850,496326.0,21785,Eagle Gate College-Boise Campus,"Private, for-profit",0,5108,Allied Health and Medical Assisting Services.,1,Undergraduate Certificate or Diploma,0.0,4.0,5,2,7,48,1,13
218851,496326.0,21785,Eagle Gate College-Boise Campus,"Private, for-profit",0,5138,"Registered Nursing, Nursing Administration, Nu...",3,Bachelor's Degree,0.0,0.0,0,1,0,100,1,0


Lets compare the two methods and see which one is better
df_cohort_filtered vs df_thresh_non_ps

In [181]:
df_cohort_filtered.shape

(51299, 25)

In [182]:
df_thresh_non_ps.shape

(53447, 17)

In [183]:
manual_cols = set(df_cohort_filtered.columns)

In [184]:
thresh_cols = set(df_thresh_non_ps.columns)

In [185]:
manual_cols.intersection(thresh_cols) # common cols

{'BBRR1_FED_COMP_N',
 'CIPCODE',
 'CIPDESC',
 'CONTROL',
 'CREDDESC',
 'CREDLEV',
 'DISTANCE',
 'EARN_COUNT_NWNE_1YR',
 'EARN_COUNT_NWNE_4YR',
 'EARN_COUNT_NWNE_5YR',
 'EARN_COUNT_NWNE_HI_1YR',
 'INSTNM',
 'IPEDSCOUNT1',
 'IPEDSCOUNT2',
 'MAIN',
 'OPEID6',
 'UNITID'}

In [186]:
manual_cols.difference(thresh_cols) # cols in manual not in thresh

{'BBRR2_FED_COMP_N',
 'BBRR3_FED_COMP_N',
 'BBRR4_FED_COMP_N',
 'DEBT_ALL_STGP_EVAL_N',
 'EARN_CNTOVER150_HI_2YR',
 'EARN_COUNT_NWNE_HI_2YR',
 'EARN_COUNT_WNE_HI_2YR',
 'EARN_MDN_HI_2YR'}

In [187]:
thresh_cols.difference(manual_cols) # cols in thresh not in manual

set()

Clearly our manual cols gives more information than threshold df, lets proceed with out manually selected and filtered df

In [188]:
df_merged = pd.merge(df_institutes_cleaned_final, df_cohort_filtered, on="UNITID", how="inner")

In [193]:
df_merged

,UNITID,INSTNM_x,MD_EARN_WNE_4YR,MD_EARN_WNE_5YR,MD_EARN_WNE_P6,GT_THRESHOLD_1YR,GT_THRESHOLD_P6,TUITIONFEE_PROG,NPT4_PRIV,MD_EARN_WNE_INC2_P6,...,EARN_COUNT_NWNE_4YR,BBRR1_FED_COMP_N,DEBT_ALL_STGP_EVAL_N,BBRR2_FED_COMP_N,BBRR4_FED_COMP_N,BBRR3_FED_COMP_N,EARN_CNTOVER150_HI_2YR,EARN_COUNT_WNE_HI_2YR,EARN_COUNT_NWNE_HI_2YR,EARN_MDN_HI_2YR
0,100654,Alabama A & M University,46562.0,52246.0,27851.0,355.0,0.4613,NaN,NaN,31228.0,...,0,14,14,16,10,10,12,12,0,47260
1,100654,Alabama A & M University,46562.0,52246.0,27851.0,355.0,0.4613,NaN,NaN,31228.0,...,0,48,45,18,16,16,6,12,0,20102
2,100654,Alabama A & M University,46562.0,52246.0,27851.0,355.0,0.4613,NaN,NaN,31228.0,...,0,46,41,26,29,29,29,31,0,52107
3,100654,Alabama A & M University,46562.0,52246.0,27851.0,355.0,0.4613,NaN,NaN,31228.0,...,0,10,11,13,12,12,12,11,0,29685
4,100654,Alabama A & M University,46562.0,52246.0,27851.0,355.0,0.4613,NaN,NaN,31228.0,...,2,14,14,15,20,20,8,11,0,50231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50223,495998,San Joaquin Valley College-Porterville,33652.0,38245.0,36007.0,3406.0,0.6090,21997.0,23531.0,41456.0,...,17,258,306,135,113,113,36,60,11,22094
50224,496265,Triangle Tech-Chambersburg,37440.0,42712.0,32045.0,63.0,0.5149,NaN,24252.0,37673.0,...,7,25,20,33,54,54,35,50,5,30409
50225,496283,Provo College-Idaho Falls Campus,33337.0,39759.0,32673.0,157.0,0.5449,NaN,31248.0,35215.0,...,3,30,22,55,35,35,9,13,0,23456
50226,496326,Eagle Gate College-Boise Campus,34674.0,38916.0,36393.0,126.0,0.6103,NaN,29546.0,NaN,...,7,48,14,117,97,96,12,17,2,25658


In [199]:
df_institutes_cleaned_final['TUITIONFEE_PROG'].isna().sum()

np.int64(4275)

In [198]:
df_merged.isna().sum().sort_values(ascending=False)

,0
TUITIONFEE_PROG,47432
NPT4_PRIV,29980
ROOMBOARD_ON,12909
MD_EARN_WNE_INC2_P6,2289
MD_EARN_WNE_INC3_P6,2289
AVGFACSAL,2023
PCT75_EARN_WNE_P11,782
GT_THRESHOLD_P6,449
GT_THRESHOLD_1YR,429
MD_EARN_WNE_P6,330


In [191]:
df_merged_clean = df_merged.dropna()
df_merged_clean

,UNITID,INSTNM_x,MD_EARN_WNE_4YR,MD_EARN_WNE_5YR,MD_EARN_WNE_P6,GT_THRESHOLD_1YR,GT_THRESHOLD_P6,TUITIONFEE_PROG,NPT4_PRIV,MD_EARN_WNE_INC2_P6,...,EARN_COUNT_NWNE_4YR,BBRR1_FED_COMP_N,DEBT_ALL_STGP_EVAL_N,BBRR2_FED_COMP_N,BBRR4_FED_COMP_N,BBRR3_FED_COMP_N,EARN_CNTOVER150_HI_2YR,EARN_COUNT_WNE_HI_2YR,EARN_COUNT_NWNE_HI_2YR,EARN_MDN_HI_2YR


In [192]:
df_merged_clean

,UNITID,INSTNM_x,MD_EARN_WNE_4YR,MD_EARN_WNE_5YR,MD_EARN_WNE_P6,GT_THRESHOLD_1YR,GT_THRESHOLD_P6,TUITIONFEE_PROG,NPT4_PRIV,MD_EARN_WNE_INC2_P6,...,EARN_COUNT_NWNE_4YR,BBRR1_FED_COMP_N,DEBT_ALL_STGP_EVAL_N,BBRR2_FED_COMP_N,BBRR4_FED_COMP_N,BBRR3_FED_COMP_N,EARN_CNTOVER150_HI_2YR,EARN_COUNT_WNE_HI_2YR,EARN_COUNT_NWNE_HI_2YR,EARN_MDN_HI_2YR


In [200]:
df_institutes_valid = df_institutes_cleaned[df_institutes_cleaned["TUITIONFEE_PROG"].notna()]

In [207]:
df_institutes_valid['INSTNM']

,INSTNM
17,New Beginning College of Cosmetology
60,Alaska Vocational Technical Center
62,Alaska Career College
63,Empire Beauty School-Tucson
64,Carrington College-Phoenix North
65,Carrington College-Mesa
66,Carrington College-Tucson
67,Arizona Academy of Beauty
68,Miller-Motte College-Arizona Automotive Institute
69,Brookline College-Phoenix


In [208]:
df_merged_with_tution = df_cohorts.merge(
    df_institutes_valid,
    on="UNITID",
    how="left"
)


In [209]:
df_merged_with_tution

,UNITID,OPEID6,INSTNM_x,CONTROL_x,MAIN_x,CIPCODE,CIPDESC,CREDLEV,CREDDESC,IPEDSCOUNT1,...,COUNT_WNE_INC3_P11,COUNT_WNE_INDEP0_P11,COUNT_WNE_INDEP1_P11,GT_THRESHOLD_P11,MD_EARN_WNE_INC1_P11,MD_EARN_WNE_INC2_P11,MD_EARN_WNE_INC3_P11,MD_EARN_WNE_INDEP0_P11,MD_EARN_WNE_INDEP1_P11,SCORECARD_SECTOR
0,100654.0,1002,Alabama A & M University,Public,1,109,Animal Sciences.,3,Bachelor's Degree,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,3,Bachelor's Degree,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,5,Master's Degree,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100654.0,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,6,Doctoral Degree,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100654.0,1002,Alabama A & M University,Public,1,199,"Agriculture, Agriculture Operations, and Relat...",3,Bachelor's Degree,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171405,496283.0,23608,Provo College-Idaho Falls Campus,"Private, for-profit",0,5138,"Registered Nursing, Nursing Administration, Nu...",3,Bachelor's Degree,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171406,496292.0,23068,Miller-Motte College-Chattanooga 2,"Private, for-profit",0,4902,Ground Transportation.,1,Undergraduate Certificate or Diploma,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171407,496326.0,21785,Eagle Gate College-Boise Campus,"Private, for-profit",0,5108,Allied Health and Medical Assisting Services.,1,Undergraduate Certificate or Diploma,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171408,496326.0,21785,Eagle Gate College-Boise Campus,"Private, for-profit",0,5138,"Registered Nursing, Nursing Administration, Nu...",3,Bachelor's Degree,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
